# 문장 유형 분류

 - 대회링크: https://dacon.io/competitions/official/236037/overview/description
 - 최고 스코어: 0.6445

### 1. 데이터 로드 및 탐색

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from transformers import TFAutoModel, AutoTokenizer
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


import warnings
warnings.filterwarnings(action='ignore')

In [2]:
file_path = 'data/'
file_name = 'train.csv'
df = pd.read_csv(file_path + file_name)
df_test = pd.read_csv(file_path + 'test.csv')

df.tail()

,ID,문장,유형,극성,시제,확실성,label
16536,TRAIN_16536,"＇신동덤＇은 ＇신비한 동물사전＇과 ＇해리 포터＇ 시리즈를 잇는 마법 어드벤처물로, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
16537,TRAIN_16537,"수족냉증은 어릴 때부터 심했으며 관절은 어디 한 곳이 아니고 목, 어깨, 팔꿈치, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
16538,TRAIN_16538,김금희 소설가는 ＂계약서 조정이 그리 어려운가 작가를 격려한다면서 그런 문구 하나 ...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
16539,TRAIN_16539,1만명이 넘는 방문자수를 기록한 이번 전시회는 총 77개 작품을 넥슨 사옥을 그대로...,사실형,긍정,과거,불확실,사실형-긍정-과거-불확실
16540,TRAIN_16540,《목민심서》의 내용이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16541 entries, 0 to 16540
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      16541 non-null  object
 1   문장      16541 non-null  object
 2   유형      16541 non-null  object
 3   극성      16541 non-null  object
 4   시제      16541 non-null  object
 5   확실성     16541 non-null  object
 6   label   16541 non-null  object
dtypes: object(7)
memory usage: 904.7+ KB


In [4]:
# ID열 제거
df.drop('ID', axis=1,inplace=True)

In [5]:
print('대화 유형 종류:', df['유형'].value_counts().to_dict())
print('대화 극성 종류:', df['극성'].value_counts().to_dict())
print('대화 시제 종류:', df['시제'].value_counts().to_dict())
print('대화 확실성 종류:', df['확실성'].value_counts().to_dict())

대화 유형 종류: {'사실형': 13558, '추론형': 2151, '대화형': 575, '예측형': 257}
대화 극성 종류: {'긍정': 15793, '부정': 565, '미정': 183}
대화 시제 종류: {'과거': 8032, '현재': 6866, '미래': 1643}
대화 확실성 종류: {'확실': 15192, '불확실': 1349}


In [6]:
# 결측치 확인
df.isna().sum()

문장       0
유형       0
극성       0
시제       0
확실성      0
label    0
dtype: int64

In [7]:
# 중복값 확인 => 31
print(df.duplicated().sum())

31


In [8]:
# 중복내용 확인
dup = df.duplicated()
dup = df[dup]['문장'].tolist()
df[df['문장'] == dup[0]]

,문장,유형,극성,시제,확실성,label
300,신용카드 빚을 뜻하는 판매신용은 2조4000억원 늘어난 91조1000억원이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
1638,신용카드 빚을 뜻하는 판매신용은 2조4000억원 늘어난 91조1000억원이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실


In [9]:
# 중복 제거
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16510 entries, 0 to 16539
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   문장      16510 non-null  object
 1   유형      16510 non-null  object
 2   극성      16510 non-null  object
 3   시제      16510 non-null  object
 4   확실성     16510 non-null  object
 5   label   16510 non-null  object
dtypes: object(6)
memory usage: 902.9+ KB


In [10]:
# label이 다르지만 같은 문장 존재
sent_dup = df[df['문장'].duplicated()]['문장'].tolist()
df[df['문장'] == sent_dup[3]]

,문장,유형,극성,시제,확실성,label
2108,직접적으로 암 덩어리를 없애거나 크기를 줄이고 암세포를 죽이기 위한 치료다.,사실형,긍정,현재,불확실,사실형-긍정-현재-불확실
15167,직접적으로 암 덩어리를 없애거나 크기를 줄이고 암세포를 죽이기 위한 치료다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실


In [11]:
sent_dup

['박근혜 정권에서 블랙리스트에 올라 지난 2014년 타의로 미국으로 떠난 이 부회장은 해외 엔터테인먼트 업계에서 지속적으로 활동해온 것으로 알려졌다.',
 '이들 게임은 국내 구글 플레이 매출 톱10 진입이 예상되는 기대작이다.',
 '각 레이스가 종료되면 ＇우마무스메＇들이 무대에 올라 눈과 귀를 즐겁게하는 공연을 펼친다.',
 '직접적으로 암 덩어리를 없애거나 크기를 줄이고 암세포를 죽이기 위한 치료다.']

In [12]:
# 문장길이 확인
sent_len = df['문장'].apply(len)
sent_len.describe()

count    16510.000000
mean        63.842520
std         35.488533
min          7.000000
25%         40.000000
50%         57.000000
75%         79.000000
max        534.000000
Name: 문장, dtype: float64

In [13]:
print(sent_len[sent_len > 80].count())
print(sent_len[sent_len > 100].count())
print(sent_len[sent_len > 120].count())
print(sent_len[sent_len > 150].count())
print(sent_len[sent_len > 200].count())
print(sent_len[sent_len > 250].count())

3959
2081
1106
443
125
31


### 2. 데이터 전처리

In [14]:
sent_type = ['사실형', '추론형', '대화형', '예측형']
sent_polarity = ['긍정', '부정', '미정']
sent_tense = ['과거', '현재', '미래']
sent_certainty = ['확실', '불확실']
df.head()

,문장,유형,극성,시제,확실성,label
0,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
1,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
2,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실
3,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
4,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실


In [15]:
asdf = '사실형-긍정-현재-확실'

features = [sent_type, sent_polarity, sent_tense, sent_certainty]
def makeLabel(label):
    sp = label.split('-')
    result = []
    for typ, feat in zip(sp, features):
        temp = [0 for _ in range(len(feat))]
        for i, f in enumerate(feat):
            if typ == f:
                temp[i] = 1
        result.append(temp)
    return result

makeLabel(asdf)

[[1, 0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0]]

In [16]:
label = df['label'].apply(makeLabel)
label[:5][0]

[[1, 0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0]]

In [17]:
MAX_LEN = 200

# 사전학습 모델
# check_point = 'klue/roberta-small'
# check_point = 'snunlp/KR-FinBert-SC'
check_point = "snunlp/KR-Medium"
# check_point = "amberoad/bert-multilingual-passage-reranking-msmarco"
# check_point = "klue/roberta-large"

tokenizer = AutoTokenizer.from_pretrained(check_point)

# train data
document = df['문장']

# target
target = df['label'].apply(makeLabel)

# x_train, x_temp, y_train, y_temp = train_test_split(document, target, test_size=0.3)
# x_valid, x_test, y_valid, y_test = train_test_split(x_temp, y_temp, test_size=0.5)

In [18]:
def bert_tokenizer(sent, max_len):
    encoded_dict = tokenizer.encode_plus(
        text = sent,
        add_special_tokens = True,      
        max_length = max_len,           
        pad_to_max_length = True,
        return_attention_mask = True,   
        truncation = True
    )
    
    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask']
    token_type_id = encoded_dict['token_type_ids']
    
    return input_id, attention_mask, token_type_id


def build_data(doc):
    x_ids = []
    x_msk = []
    x_typ = []

    for sent in tqdm(doc):
        input_id, attention_mask, token_type_id = bert_tokenizer(sent, MAX_LEN)
        x_ids.append(input_id)
        x_msk.append(attention_mask)
        x_typ.append(token_type_id)

    x_ids = np.array(x_ids, dtype=int)
    x_msk = np.array(x_msk, dtype=int)
    x_typ = np.array(x_typ, dtype=int)

    return x_ids, x_msk, x_typ

In [20]:
def make_labels(dataset):
    types = []
    polars = []
    tenses = []
    certains = []
    for i, data in enumerate(dataset):
        types.append(data[0])
        polars.append(data[1])
        tenses.append(data[2])
        certains.append(data[3])
    
    return np.array(types), np.array(polars), np.array(tenses), np.array(certains)

In [21]:
# All data
x_train_ids, x_train_msk, x_train_typ = build_data(document)
x_train = [x_train_ids, x_train_msk, x_train_typ]

types, polars, tenses, certains = make_labels(target)

100%|██████████████████████████████████████████████████████████████████████████| 16510/16510 [00:02<00:00, 6258.73it/s]


### 3. 모델 정의 및 훈련

In [24]:
bert = TFAutoModel.from_pretrained(check_point, from_pt=True)
bert.trainable = False

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [35]:
# BERT 입력
# ---------
x_input_ids = Input(batch_shape = (None, MAX_LEN), dtype = tf.int32)
x_input_msk = Input(batch_shape = (None, MAX_LEN), dtype = tf.int32)
x_input_typ = Input(batch_shape = (None, MAX_LEN), dtype = tf.int32)

# BERT 출력
# [0]: (None, 60, 768) - sequence_output, 
# [1]: (None, 768) - pooled_output (모델의 맨 마지막 layer의 CLS토큰 벡터)
# ------------------------------------------------------------------------
output_bert = bert([x_input_ids, x_input_msk, x_input_typ])[1]

type_drop = Dropout(0.2)(output_bert)
type_out = Dense(4, activation = 'softmax')(type_drop)
polar_drop = Dropout(0.2)(output_bert)
polar_out = Dense(3, activation = 'softmax')(polar_drop)
tense_drop = Dropout(0.2)(output_bert)
tense_out = Dense(3, activation = 'softmax')(tense_drop)
certain_drop = Dropout(0.2)(output_bert)
certain_out = Dense(2, activation = 'softmax')(certain_drop)

type_out_model = Model([x_input_ids, x_input_msk, x_input_typ], type_out)
type_out_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01))

polar_out_model = Model([x_input_ids, x_input_msk, x_input_typ], polar_out)
polar_out_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01))

tense_out_model = Model([x_input_ids, x_input_msk, x_input_typ], tense_out)
tense_out_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01))

certain_out_model = Model([x_input_ids, x_input_msk, x_input_typ], certain_out)
certain_out_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01))

In [36]:
type_out_model.fit(x_train, types, batch_size=32, epochs=3)
polar_out_model.fit(x_train, polars, batch_size=32, epochs=3)
tense_out_model.fit(x_train, tenses, batch_size=32, epochs=3)
certain_out_model.fit(x_train, certains, batch_size=32, epochs=3)

Epoch 1/3
516/516 [==============================] - 205s 372ms/step - loss: 0.4791
Epoch 2/3
516/516 [==============================] - 193s 375ms/step - loss: 0.4527
Epoch 3/3
516/516 [==============================] - 192s 372ms/step - loss: 0.4573
Epoch 1/3
516/516 [==============================] - 208s 378ms/step - loss: 0.1904
Epoch 2/3
516/516 [==============================] - 195s 379ms/step - loss: 0.1916
Epoch 3/3
516/516 [==============================] - 197s 382ms/step - loss: 0.1959
Epoch 1/3
516/516 [==============================] - 211s 379ms/step - loss: 0.5934
Epoch 2/3
516/516 [==============================] - 202s 392ms/step - loss: 0.5620
Epoch 3/3
516/516 [==============================] - 191s 370ms/step - loss: 0.5723
Epoch 1/3
516/516 [==============================] - 208s 376ms/step - loss: 0.2970
Epoch 2/3
516/516 [==============================] - 194s 375ms/step - loss: 0.3051
Epoch 3/3
516/516 [==============================] - 199s 385ms/step - loss:

### 4. 모델 성능분석

In [37]:
# 예측결과를 갖고 가장 높은 확률을 갖는 부분을 1로 리턴
def compute_result(pred):
    result = [[0 for _ in range(len(pred[0]))] for _ in range(len(pred))]
    for i, p in enumerate(pred):
        max_idx = p.argmax()
        result[i][max_idx] = 1
    return result

# 정확도 계산
def compute_accuracy(real, pred):
    total = len(real)
    cnt = 0
    for lb, pd in zip(real, pred):
        for l, p in zip(lb, pd):
            if l != p:
                break
        else:
            cnt += 1
            
    return cnt / total

In [38]:
# 성능 테스트
# models = [type_out_model, polar_out_model, tense_out_model, certain_out_model]
# labels = [l_type_test, l_polar_test, l_tense_test, l_certain_test]
# result = []
# for md, lb in zip(models, labels):
#     pred = md.predict(x_test)
#     temp_result = compute_result(pred)
#     acc = compute_accuracy(lb, temp_result)
#     result.append(round(acc, 4))
    
# result

# # [0.7734, 0.9552, 0.3728, 0.895]
# # [0.8207, 0.9544, 0.4176, 0.9055]

### 5. 결과제출

In [39]:
df_test.head()
test_doc = df_test['문장']
x_tt_ids, x_tt_msk, x_tt_typ = build_data(test_doc)
x_tt = [x_tt_ids, x_tt_msk, x_tt_typ]

100%|████████████████████████████████████████████████████████████████████████████| 7090/7090 [00:01<00:00, 5153.19it/s]


In [40]:
models = [type_out_model, polar_out_model, tense_out_model, certain_out_model]
predict = []
for md in models:
    pred = md.predict(x_tt)
    result = compute_result(pred)
    predict.append(result)

222/222 [==============================] - 76s 324ms/step


In [41]:
# sent_type = ['사실형', '추론형', '대화형', '예측형']
# sent_polarity = ['긍정', '부정', '미정']
# sent_tense = ['과거', '현재', '미래']
# sent_certainty = ['확실', '불확실']

ans_type = {
    0: '사실형',
    1: '추론형',
    2: '대화형',
    3: '예측형'
}
ans_polar = {
    0: '긍정',
    1: '부정',
    2: '미정'
}
ans_tense = {
    0: '과거',
    1: '현재',
    2: '미래'
}
ans_certain = {
    0: '확실',
    1: '불확실'
}

answer = []
ans_list = [ans_type, ans_polar, ans_tense, ans_certain]
for pred, ans in zip(predict, ans_list):
    temp_ans = []
    for p in pred:
        for i, e in enumerate(p):
            if e == 0:
                continue
            idx = i
            temp_ans.append(ans[idx])
            break
    answer.append(temp_ans)

In [42]:
answer = np.array(answer)
answer

array([['사실형', '사실형', '사실형', ..., '사실형', '사실형', '사실형'],
       ['긍정', '긍정', '긍정', ..., '긍정', '긍정', '긍정'],
       ['현재', '현재', '과거', ..., '현재', '미래', '과거'],
       ['확실', '확실', '확실', ..., '확실', '불확실', '확실']], dtype='<U3')

In [43]:
submit = []
for i in range(len(answer[0])):
    result = '-'.join(answer[:, i].tolist())
    submit.append(result)

In [44]:
submit[:5]

['사실형-긍정-현재-확실',
 '사실형-긍정-현재-확실',
 '사실형-긍정-과거-확실',
 '사실형-긍정-과거-확실',
 '사실형-긍정-과거-확실']

In [45]:
df_submit = df_test
df_submit['label'] = submit
df_submit = df_submit.drop(['문장'], axis=1)
df_submit.head()

,ID,label
0,TEST_0000,사실형-긍정-현재-확실
1,TEST_0001,사실형-긍정-현재-확실
2,TEST_0002,사실형-긍정-과거-확실
3,TEST_0003,사실형-긍정-과거-확실
4,TEST_0004,사실형-긍정-과거-확실


In [46]:
df_submit.to_csv('submit_medium_drop.csv', index=False)